## 💹 시계열 데이터 분석 및 예측 모델링 연습 과제 ✍️

이 노트북은 학생들이 시계열 예측 모델링의 각 단계를 직접 코드로 구현해보는 실습을 위해 제작되었습니다. 각 모델에 대한 이론적 설명을 읽고, 바로 아래 코드 셀에 지시에 따라 직접 코드를 작성해보세요.

### 📝 **파일 정보**

  * **원본 노트북:** Time\_series\_forecasting\_💹\_(ES,\_ARIMA,\_VAR).ipynb
  * **저자:** Enes Zvornicanin
  * **목표:** 주요 시계열 예측 모델(이동 평균, 지수 평활, ARIMA, VAR 등)의 개념을 이해하고 파이썬으로 직접 구현하는 능력을 기릅니다.

-----

### **Part 1: 계량 경제학적 방법**

#### **초급 - 중급 수준**

### **소개**

이 노트북의 목적은 시계열 예측을 위한 여러 모델에 대한 간단하고 명확한 이론적 설명과 최소한의 작동 예제를 제공하는 것입니다. \<u\>참고: 주된 목표는 특정 데이터셋에서 최상의 예측을 하는 것이 아니라, 시계열 예측을 위한 알고리즘들을 나열하고 설명하는 것입니다.\</u\> 따라서 이 노트북에서 월별 판매량을 예측하기 위해 일부 방법을 사용하는 것은 큰 의미가 없을 수 있으며 예시로만 제공됩니다.

**사전 지식:** Pandas 및 시계열에 대한 기본 지식.

-----

### **목차**

  * [자기상관 (ACF)](#topic7)
  * [부분 자기상관 (PACF)](#topic8)
  * [정상성 (Stationarity)](#topic9)
      * [증강 디키-풀러 검정 (ADF)](#topic9.1)
      * [Kwiatkowski-Phillips-Schmidt-Shin 검정 (KPSS)](#topic9.2)
  * [Naive 모델](#topic1)
  * [단순 이동 평균](#topic2)
  * [가중 이동 평균](#topic3)
  * [단순 지수 평활 (SES)](#topic4)
  * [이중 지수 평활 (Holt's)](#topic5)
  * [삼중 지수 평활 (Holt-Winters)](#topic6)
  * [자기회귀 모델 (AR)](#topic10)
      * [AIC와 BIC](#topic10.1)
  * [이동 평균 모델 (MA)](#topic11)
  * [자기회귀 이동 평균 모델 (ARMA)](#topic12)
  * [자기회귀 누적 이동 평균 모델 (ARIMA)](#topic13)
  * [계절성 ARIMA (SARIMA)](#topic14)
  * [외생 변수가 있는 계절성 ARIMA (SARIMAX)](#topic15)
  * [벡터 자기회귀 모델 (VAR)](#topic16)
      * [공적분 (Cointegration)](#topic16.1)
      * [벡터 오차 수정 모델 (VECM)](#topic17)
  * [벡터 자기회귀 이동 평균 모델 (VARMA)](#topic18)
  * [외생 변수가 있는 벡터 자기회귀 이동 평균 모델 (VARMAX)](#topic19)

-----

### **데이터 준비**

In [2]:
import pandas as pd
df = pd.read_csv('../datasets/tsa/kaggle/Predict-Future-Sales/sales_train.csv')
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [ ]:
# [실습 1] 필요한 라이브러리를 임포트하고 데이터를 불러온 후, 주별 판매량으로 데이터를 집계하세요.
# - 라이브러리: numpy, pandas, matplotlib.pyplot
# - 데이터 경로: '../datasets/tsa/kaggle/Predict-Future-Sales/sales_train.csv'
# - 'date'와 'item_cnt_day' 열만 사용합니다.
# - 'date' 열을 datetime 형식으로 변환하고 인덱스로 설정합니다.
# - 데이터를 주(Weekly) 단위로 그룹화하고 'item_cnt_day'의 합계를 구합니다.
# - 열 이름을 'weekly_sales'로 변경하고, 시각화하여 확인합니다.

# 여기에 코드를 작성하세요.

-----
### **자기상관 (ACF)**

시계열 데이터와 자체의 지연된(lagged) 버전 간의 **단순 피어슨 상관관계**입니다.

아래 ACF 플롯의 파란색 막대는 오차 범위를 나타내며, 이 막대 안에 있는 것은 통계적으로 유의미하지 않습니다. 즉, 이 영역 밖의 상관관계 값은 통계적 우연이 아닌 실제 상관관계일 가능성이 매우 높다는 것을 의미합니다. 신뢰 구간은 기본적으로 95%로 설정됩니다.

lag 0의 경우 ACF는 항상 1입니다.

In [ ]:
# [실습 2] statsmodels 라이브러리의 plot_acf를 사용하여 'weekly_sales'의 자기상관을 시각화하세요.
# - lags는 70으로 설정합니다.
# - x축 레이블은 'lag', y축 레이블은 'correlation'으로 지정하세요.

# 여기에 코드를 작성하세요.

-----

<a id="topic8"></a>

### **부분 자기상관 (PACF)**

시계열 데이터와 자체의 지연된 버전 간의 **"직접적인" 상관관계**입니다.

예를 들어, lag 2에 대한 신호 S의 ACF를 생각해보겠습니다. $S\_{t-2}$와 $S\_{t}$ 사이의 상관관계를 계산할 때, 이는 $S\_{t-2}$가 $S\_{t}$에 미치는 직접적인 영향뿐만 아니라 $S\_{t-1}$을 통한 간접적인 영향도 포함합니다. PACF로 정말 계산하고 싶은 것은 간접적인 요소를 제외한 $S\_{t-2}$가 $S\_{t}$에 미치는 직접적인 영향뿐입니다.

lag 2에 대한 신호 $S$의 PACF는 회귀 모델 $S\_{t} = \\phi\_{21}S\_{t-1} + \\phi\_{22}S\_{t-2} + \\epsilon\_{t}$에서 계수 $\\phi\_{22}$를 PACF 값으로 사용하여 계산됩니다.

In [ ]:
# [실습 3] statsmodels 라이브러리의 plot_pacf를 사용하여 'weekly_sales'의 부분 자기상관을 시각화하세요.
# - lags는 70으로 설정합니다.
# - x축 레이블은 'lag', y축 레이블은 'pacf coefficient'로 지정하세요.

# 여기에 코드를 작성하세요.

-----
### **정상성 (Stationarity)**

시계열 예측에서 정상성은 대부분의 알고리즘이 요구하는 가장 중요한 조건 중 하나입니다. 간단히 말해, 시계열 $S\_{t}$가 정상성(약한 의미의 정상성)을 만족하려면 다음 조건들이 충족되어야 합니다.

1.  $S\_{t}$의 평균이 일정해야 합니다.
2.  $S\_{t}$의 표준편차가 일정해야 합니다.
3.  $S\_{t}$에 계절성이 없어야 합니다.

신호의 정상성은 시각적으로 확인하거나(근사치), 더 정확한 답을 위해 통계적 가설 검정을 사용할 수 있습니다. 이를 위해 두 가지 검정을 사용합니다:

  * **증강 디키-풀러 검정 (ADF)**

      * 귀무가설: 신호는 비정상성이다.
      * p-value \< 0.05 이면, 귀무가설을 기각하고 신호가 정상성이라고 판단합니다.

  * **Kwiatkowski-Phillips-Schmidt-Shin 검정 (KPSS)**

      * 귀무가설: 신호는 정상성이다.
      * p-value \< 0.05 이면, 귀무가설을 기각하고 신호가 비정상성이라고 판단합니다.

신호 $S\_{t}$가 비정상성일 경우, 차분($T\_{t} = S\_{t} - S\_{t-1}$)이나 변화율($T\_{t} =\\frac{S\_{t} - S\_{t-1}}{S\_{t-1}}$)을 계산하여 정상 신호 $T\_{t}$로 변환할 수 있습니다.

In [ ]:
# [실습 4] 원본 'weekly_sales' 데이터, 1차 차분 데이터, 변화율 데이터 각각에 대해 정상성 검정(ADF, KPSS)을 수행하고 ACF 플롯과 함께 시각화하세요.
# - 주어진 adf_stat, kpss_stat, plot_acf_stat_test 함수를 활용하여 3x2 subplot에 결과를 표시합니다.
# - 첫 번째 행: 원본 데이터
# - 두 번째 행: 1차 차분 데이터 (diff() 사용)
# - 세 번째 행: 변화율 데이터 (pct_change() 사용, 0으로 나누는 오류 방지를 위해 0을 5000으로 대체)

# adf_stat, kpss_stat, plot_acf_stat_test 함수 정의 (이 부분은 수정하지 않아도 됩니다)
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf

def adf_stat(time_series):
    result = adfuller(time_series.values)
    s = ''
    s+= 'ADF Statistic: %.3f\\n' % result[0]
    s+= 'p-value: %.3f\\n' % result[1]
    s+= 'Critical Values:\\n'
    for key, value in result[4].items():
        s += '%s: %.3f\\n' % (key, value)
    return s

def kpss_stat(time_series):
    import warnings
    warnings.filterwarnings("ignore")
    result = kpss(time_series.values)
    s = ''
    s+= 'KPSS Statistic: %.3f\\n' % result[0]
    s+= 'p-value: %.3f\\n' % result[1]
    s+= 'Critical Values:\\n'
    for key, value in result[3].items():
        if key == '2.5%':
            continue
        s += '%s: %.3f\\n' % (key, value)
    return s

# 여기에 시각화 코드를 작성하세요.
# 힌트: plt.subplots(3, 2)를 사용하여 axes를 만들고, plot_acf_stat_test 함수를 각 데이터에 맞게 호출합니다.

-----
### **Naive Forecast 모델**

다음 시점의 값은 이전 시점의 값과 동일하다고 예측합니다.

$$Y_{t} = Y_{t-1}$$

In [ ]:
# [실습 5] Naive 모델을 사용하여 마지막 50주간의 판매량을 예측하고 실제 값과 비교하여 시각화하세요.
# - 예측값('weekly_sales_pred')은 이전 시점의 'weekly_sales' 값입니다. (shift() 사용)
# - 예측값의 50개 구간 롤링 표준편차를 구해 신뢰구간을 계산하고 함께 표시하세요. (rolling(50).std())
# - 신뢰구간: 예측값 ± 2 * 표준편차
# - 평균 제곱 오차(MSE)를 계산하여 제목에 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **단순 이동 평균 (Simple Moving Average)**

신호 $y\_{t}$에 대해, 단순 이동 평균은 마지막 $k$개 항목의 평균입니다.

$$y_{t} = \frac{y_{t-1} + y_{t-2} +...+y_{t-k}}{k}, t\geq k$$

이 방법은 일반적으로 신호의 정상성을 가정하지만, 짧은 윈도우(k)를 사용하면 비정상 신호에도 잘 작동할 수 있습니다.

In [ ]:
# [실습 6] 단순 이동 평균(k=5)을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 예측값은 이전 시점 값들의 롤링 평균입니다. (shift() 후에 rolling(5).mean() 사용)
# - 신뢰구간과 MSE를 계산하여 함께 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **가중 이동 평균 (Weighted Average)**

이 방법은 지연된 값에 가중치를 곱하여 이동 평균을 개선한 것입니다.

$$y_{t} = w_{1}y_{t-1} + w_{2}y_{t-2} +...+w_{k}y_{t-k}, \quad \text{여기서} \sum_{i=1}^{k}w_{i} = 1$$

In [ ]:
# [실습 7] 가중 이동 평균(k=10)을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 비정상성 문제를 해결하기 위해 'weekly_sales'의 1차 차분 데이터에 적용합니다.
# - 가중치(wts)는 1부터 k까지의 선형 배열을 정규화하여 사용합니다.
# - 예측 후, 원래 스케일로 복원하여 실제 값과 비교합니다.
# - 신뢰구간과 MSE를 계산하여 함께 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **단순 지수 평활 (Simple Exponential Smoothing, SES)**

이 방법은 이전 예측과 해당 예측의 이전 오차 사이의 선형 조합으로 시계열을 예측합니다.

$$\hat{y}_{t} = \alpha y_{t-1} + (1-\alpha)\hat{y}_{t-1}$$

여기서 $\\hat{y}*{t}$는 시간 $t$에서의 예측값, $y*{t}$는 실제 값, $\\alpha \\in [0,1]$는 평활 상수입니다. 이 방법은 추세나 계절성이 없는 신호에 사용됩니다.

In [ ]:
# [실습 8] 단순 지수 평활(SES)을 사용하여 마지막 100주간의 판매량을 예측하고 시각화하세요.
# - 1차 차분된 데이터에 SES를 적용합니다.
# - for 루프를 사용하여 한 번에 한 스텝씩 예측을 수행합니다 (롤링 예측).
# - statsmodels.tsa.holtwinters의 SimpleExpSmoothing을 사용합니다.
# - 예측 후, 원래 스케일로 복원하여 마지막 50주간의 결과를 실제 값과 비교합니다.
# - 신뢰구간과 MSE를 계산하여 함께 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **이중 지수 평활 (Double Exponential Smoothing, Holt's)**

Holt의 지수 평활은 추세가 있는 데이터의 예측을 허용하는 단순 지수 평활의 확장 버전입니다. 이 방법은 수준(level)과 추세(trend)에 대한 두 개의 평활 방정식을 추정합니다.

$$
\begin{align*}
\text{예측 방정식} & \quad \hat{y}_t = l_{t-1} + b_{t-1} \\
\text{수준 방정식} & \quad l_{t-1} = \alpha y_{t-1} + (1-\alpha)(l_{t-2} + b_{t-2}) \\
\text{추세 방정식} & \quad b_{t-1} = \beta(l_{t-1} - l_{t-2}) + (1-\beta)b_{t-2}
\end{align*}
$$

이 방법은 추세는 있지만 계절성이 없는 신호에 사용됩니다.

In [ ]:
# [실습 9] 이중 지수 평활(Holt's)을 사용하여 마지막 100주간의 판매량을 예측하고 시각화하세요.
# - 원본 데이터('weekly_sales')에 직접 적용합니다.
# - for 루프를 사용하여 롤링 예측을 수행합니다.
# - statsmodels.tsa.holtwinters의 ExponentialSmoothing을 사용하고, trend='add'로 설정합니다.
# - 마지막 50주간의 결과를 실제 값과 비교합니다.
# - 신뢰구간과 MSE를 계산하여 함께 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **삼중 지수 평활 (Triple Exponential Smoothing, Holt-Winters)**

HWES는 이중 지수 평활 방법에 계절성 요소(방정식)를 추가한 확장 버전입니다. 즉, **추세와 계절성이 있는 신호**에 적용할 수 있습니다.

$$
\begin{align*}
\text{예측 방정식} & \quad \hat{y}_{t+1} = l_{t} + b_{t} + s_{t-m} \\
\text{수준 방정식} & \quad l_{t} = \alpha (y_{t}-s_{t-m}) + (1-\alpha)(l_{t-1} + b_{t-1}) \\
\text{추세 방정식} & \quad b_{t} = \beta(l_{t} - l_{t-1}) + (1-\beta)b_{t-1} \\
\text{계절 방정식} & \quad s_{t} = \gamma(y_{t} - l_{t-1} - b_{t-1}) + (1-\gamma)s_{t-m}
\end{align*}
$$

In [ ]:
# [실습 10] 삼중 지수 평활(Holt-Winters)을 사용하여 마지막 100주간의 판매량을 예측하고 시각화하세요.
# - 원본 데이터('weekly_sales')에 직접 적용합니다.
# - for 루프를 사용하여 롤링 예측을 수행합니다.
# - ExponentialSmoothing을 사용하고, trend='add', seasonal='add', seasonal_periods=12로 설정합니다.
# - 마지막 50주간의 결과를 실제 값과 비교합니다.
# - 신뢰구간과 MSE를 계산하여 함께 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **자기회귀 모델 (AR)**

신호 $S\_{t}$가 자신의 과거 값에만 의존하는 모델입니다. 예를 들어, AR(3)는 과거 3개의 값에 의존하며 다음과 같이 쓸 수 있습니다.
$$S_{t} =\beta_{0} + \beta_{1}S_{t-1} + \beta_{2}S_{t-2} + \beta_{3}S_{t-3} + \epsilon_{t}$$
AR 모델은 정상성을 요구합니다. AR(p) 모델의 차수 p는 PACF 플롯에서 유의미한 스파이크를 기반으로 선택할 수 있습니다.

In [ ]:
# [실습 11] AR(3) 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 1차 차분된 데이터('weekly_sales_diff')에 AR(3) 모델을 적용합니다.
# - statsmodels.tsa.arima.model의 ARIMA를 사용하고, order=(3, 0, 0)으로 설정합니다.
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **이동 평균 모델 (MA)**

MA는 과거의 오차 항(error terms)에 의존하는 모델입니다. 예를 들어, 신호 $S\_{t}$에 대한 MA(3)는 다음과 같이 공식화할 수 있습니다.
$$S_{t} = \mu + \epsilon_{t} + \gamma_{1}\epsilon_{t-1}+\gamma_{2}\epsilon_{t-2}+\gamma_{3}\epsilon_{t-3}$$
MA 모델은 정상성을 요구합니다. MA(q) 모델의 차수 q는 ACF 플롯이 lag q 이후에 급격히 감소하는 경우 선택할 수 있습니다.

In [ ]:
# [실습 12] MA(4) 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 1차 차분된 데이터('weekly_sales_diff')에 MA(4) 모델을 적용합니다.
# - ARIMA 클래스를 사용하고, order=(0, 0, 4)로 설정합니다.
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **자기회귀 이동 평균 모델 (ARMA)**

ARMA(p,q)는 AR(p)와 MA(q) 모델의 조합입니다.
$$S_{t} =\beta_{0} + \beta_{1}S_{t-1} + ... + \beta_{p}S_{t-p} + \epsilon_{t} + \gamma_{1}\epsilon_{t-1}+...+ \gamma_{q}\epsilon_{t-q}$$
ARMA 모델은 정상성을 요구합니다.

In [ ]:
# [실습 13] ARMA(3, 3) 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 1차 차분된 데이터('weekly_sales_diff')에 ARMA(3, 3) 모델을 적용합니다.
# - ARIMA 클래스를 사용하고, order=(3, 0, 3)으로 설정합니다.
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **자기회귀 누적 이동 평균 모델 (ARIMA)**

ARIMA(p,d,q) 모델은 시계열을 정상성으로 만드는 I(d) 기능이 추가된 ARMA(p,q) 모델입니다. 여기서 d는 차분 횟수를 의미합니다. 이 모델은 비정상 시계열에 직접 적용할 수 있습니다.

In [ ]:
# [실습 14] ARIMA(3, 1, 3) 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 원본 비정상 데이터('weekly_sales')에 ARIMA(3, 1, 3) 모델을 직접 적용합니다.
# - ARIMA 클래스를 사용하고, order=(3, 1, 3)으로 설정합니다. (d=1이 차분을 의미)
# - for 루프를 사용하여 롤링 예측을 수행합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **계절성 ARIMA (SARIMA)**

SARIMA(p, d, q)(P, D, Q)m은 ARIMA(p, d, q)의 확장입니다. 새로운 매개변수 P, D, Q는 각각 계절적 자기회귀, 누적, 이동 평균 차수를 의미합니다. m은 단일 계절 기간에 있는 시간 단계의 수를 의미합니다.

아마도 가장 좋은 방법은 미리 정의된 SARIMA 하이퍼파라미터 세트를 사용하여 그리드 검색을 적용하거나, AIC를 기반으로 최상의 모델을 선택하는 `auto_arima`를 사용하는 것입니다.

In [ ]:
# [실습 15] pmdarima 라이브러리의 auto_arima를 사용하여 최적의 SARIMA 모델을 찾고, 이를 이용해 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 원본 데이터('weekly_sales')에 auto_arima를 적용합니다.
# - for 루프를 사용하여 롤링 예측을 수행합니다. (auto_arima는 루프 안에서 매번 최적 모델을 찾습니다)
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# pmdarima 설치 (필요시)
# !pip install pmdarima

# 여기에 코드를 작성하세요.

-----
### **외생 변수가 있는 계절성 ARIMA (SARIMAX)**

SARIMAX 모델은 외생 변수(exogenous variables), 즉 공변량(covariates)을 추가하는 SARIMA 모델의 확장입니다. 외생 변수는 예측 대상 시계열과 동일한 시간 단계에서 관측값을 갖는 추가적인 요인 또는 입력 특성을 나타냅니다.

$$y_{t} = \beta_{0} + \beta_{1}x_{t} + u_{t}$$
$$(1-\phi_{1}B)...(1-B)^{d}...u_{t}=(1+\theta_{1}B)... \epsilon_{t}$$

여기서 첫 번째 방정식은 선형 회귀이고, 두 번째 방정식은 SARIMA 과정을 설명합니다.

In [ ]:
# [실습 16] SARIMAX 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 외생 변수(exogenous variable)로 '최근 3주간 판매량의 표준편차'를 생성합니다. (rolling(3).std().shift())
# - auto_arima를 사용하되, X 인자에 외생 변수 데이터를 전달합니다.
# - for 루프를 사용하여 롤링 예측을 수행합니다. 예측 시점의 외생 변수 값도 함께 전달해야 합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **벡터 자기회귀 모델 (VAR)**

AR 모델과 유사하지만 여러 시계열을 사용합니다. 각 신호가 자신의 과거뿐만 아니라 다른 신호의 이전 값에도 의존하는 다변량 시계열 모델입니다. 두 시계열 $y\_{t}$와 $x\_{t}$를 사용하는 가장 간단한 VAR(1) 모델은 다음과 같습니다.
$$y_{t} = \alpha_{1} + \beta_{11}y_{t-1} + \beta_{12}x_{t-1} + \epsilon_{y, t}$$
$$x_{t} = \alpha_{2} + \beta_{21}y_{t-1} + \beta_{22}x_{t-1} + \epsilon_{x, t}$$
VAR 모델은 신호의 정상성을 요구합니다.

In [ ]:
# [실습 17] VAR 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 두 개의 정상 시계열: 'weekly_sales_diff'와 'weekly_sales_rolling_std_3'을 사용합니다.
# - statsmodels.tsa.api의 VAR를 사용합니다. 모델은 AIC를 기반으로 최적의 lag p를 선택합니다.
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **벡터 오차 수정 모델 (VECM)**

신호 $x\_{t}$와 $y\_{t}$가 **공적분(cointegrated)** 관계에 있을 경우, VAR 모델 대신 VEC 모델을 사용하는 것이 권장됩니다. 공적분 관계는 두 비정상 시계열의 선형 조합이 정상 시계열이 되는 경우를 말하며, 이는 두 변수 간에 장기적인 균형 관계가 존재함을 의미합니다. VECM은 이러한 장기 균형 관계에서 벗어나는 단기적 변동을 모델링합니다.

$$\Delta y_{t} = \beta_{1}\Delta x_{t} + \psi(y_{t-1} - \pi_{1} -\pi_{2}x_{t-1}) + \epsilon_{t}$$

여기서 $\\psi(y\_{t-1} - \\dots)$ 항이 오차 수정 메커니즘을 나타냅니다.

In [ ]:
# [실습 18] VECM을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 공적분 관계에 있는 두 신호: 'weekly_sales'와 'weekly_sales_plus_noise'를 생성하여 사용합니다.
#   ('weekly_sales_plus_noise'는 'weekly_sales'에 임의의 노이즈를 더한 것입니다.)
# - statsmodels.tsa.api의 VECM을 사용합니다. (coint_rank=1, k_ar_diff=5)
# - for 루프를 사용하여 롤링 예측을 수행합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **벡터 자기회귀 이동 평균 모델 (VARMA)**

이 모델은 VAR과 MA 모델의 조합입니다. 가장 간단한 형태인 VARMA(1, 1)는 다음과 같이 공식화됩니다.
$$y_{t} = \alpha_{1} + \beta_{11}y_{t-1} + \beta_{12}x_{t-1} + \epsilon_{y, t}+\gamma_{11} \epsilon_{y, t-1}$$
$$x_{t} = \alpha_{2} + \beta_{21}y_{t-1} + \beta_{22}x_{t-1} + \epsilon_{x, t}+\gamma_{21} \epsilon_{x, t-1}$$
VARMA는 정상성을 요구합니다.

In [ ]:
# [실습 19] VARMA 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 두 개의 정상 시계열: 'weekly_sales_diff'와 'weekly_sales_rolling_std_3'을 사용합니다.
# - statsmodels.tsa.api의 VARMAX를 사용합니다. (MA 부분이 있으므로 VARMAX 사용)
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.

-----
### **외생 변수가 있는 벡터 자기회귀 이동 평균 모델 (VARMAX)**

VARMAX는 외생 변수가 있는 VARMA 모델입니다.
$$y_{t} = \alpha_{1} + \beta_{11}y_{t-1} + \beta_{12}x_{t-1} +\delta_{11}z_{t} + \epsilon_{y, t}+\gamma_{11} \epsilon_{y, t-1}$$
$$x_{t} = \alpha_{2} + \beta_{21}y_{t-1} + \beta_{22}x_{t-1} +\delta_{21}z_{t} + \epsilon_{x, t}+\gamma_{21} \epsilon_{x, t-1}$$
VARMAX는 정상성을 요구합니다.

In [ ]:
# [실습 20] VARMAX 모델을 사용하여 마지막 50주간의 판매량을 예측하고 시각화하세요.
# - 두 개의 정상 시계열: 'weekly_sales_diff'와 'weekly_sales_rolling_std_3'을 사용합니다.
# - 외생 변수로 'weekly_sales_diff_plus_noise'를 생성하여 사용합니다.
# - statsmodels.tsa.statespace.varmax의 VARMAX를 사용합니다.
# - for 루프를 사용하여 롤링 예측을 수행하고, 결과를 원래 스케일로 복원합니다.
# - 마지막 50주간의 결과를 실제 값과 비교하고, 신뢰구간과 MSE를 표시하세요.

# 여기에 코드를 작성하세요.